In [111]:
import os
import zipfile
import glob


for file in glob.glob('zip_dataset/*.zip'):
  print(file)
  with zipfile.ZipFile(file, 'r') as zip_ref:
    zip_ref.extractall('metadata')

zip_dataset\od_17191_produksi_ubi_kayu_berdasarkan_kabupatenkota.zip
zip_dataset\od_17192_produksi_ubi_jalar_berdasarkan_kabupatenkota.zip
zip_dataset\od_17232_luas_areal_tanam_kacang_hijau_berdasarkan_kabupatenkota.zip
zip_dataset\od_17235_luas_areal_tanam_ubi_kayu_berdasarkan_kabupatenkota.zip
zip_dataset\od_17236_luas_areal_tanam_ubi_jalar_berdasarkan_kabupatenkota.zip
zip_dataset\od_17238_luas_areal_tanam_kedelai_berdasarkan_kabupatenkota.zip
zip_dataset\od_17239_luas_areal_tanam_kacang_tanah_berdasarkan_kabupatenkota.zip
zip_dataset\od_17242_luas_panen_kedelai_berdasarkan_kabupatenkota.zip
zip_dataset\od_17243_luas_panen_kacang_tanah_berdasarkan_kabupatenkota.zip
zip_dataset\od_17244_luas_panen_kacang_hijau_berdasarkan_kabupatenkota.zip
zip_dataset\od_17245_luas_panen_ubi_kayu_berdasarkan_kabupatenkota.zip
zip_dataset\od_17246_luas_panen_ubi_jalar_berdasarkan_kabupatenkota.zip
zip_dataset\od_17260_produksi_kedelai_berdasarkan_kabupatenkota.zip
zip_dataset\od_17261_produksi_kacang_

In [112]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, classification_report

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [113]:
def wrangling_data(dataset_type: str):
  wragled_data = pd.DataFrame()

  for folder in os.listdir('metadata'):
    if os.path.isdir(os.path.join('metadata', folder)):
      for file in os.listdir(f'metadata/{folder}'):
        if dataset_type in file and file.endswith('.csv'):
          data = pd.read_csv(f'metadata/{folder}/{file}')

          if dataset_type in ('produksi',):
            file_name = '_'.join(file.split('_')[3:-3])
            data = data[data['tahun'] >= 2015].drop_duplicates(['nama_kabupaten_kota', 'tahun'], keep='last')
            data = data.rename(
              columns={col: col.split('_')[0] + '_' + file_name for col in data.columns if col.startswith('produksi')})
          
          if dataset_type in ('luas_areal_tanam', 'luas_panen'):
            col_name = '_'.join(file.split("_")[2:-3])
            data.rename(columns={data.columns[5]: col_name}, inplace=True)

          if wragled_data.empty:
            wragled_data = data
            col_data = wragled_data.iloc[:, 5:6]
            wragled_data = wragled_data.drop(col_data, axis=1)
            wragled_data = pd.concat([wragled_data.reset_index(drop=True), col_data.reset_index(drop=True)], axis=1)
          else:
            column_name = [col for col in data.columns if dataset_type in col][0]
            data = data[[column_name]]
            wragled_data = pd.concat([wragled_data.reset_index(drop=True), data.reset_index(drop=True)], axis=1)

  return wragled_data

In [114]:
dataset_produktivitas = wrangling_data('produktivitas')
dataset_produktivitas

,id,kode_provinsi,nama_provinsi,kode_kabupaten_kota,nama_kabupaten_kota,satuan,tahun,produktivitas_kedelai,produktivitas_kacang_tanah,produktivitas_kacang_hijau,produktivitas_ubi_kayu,produktivitas_ubi_jalar
0,1,32,JAWA BARAT,3201,KABUPATEN BOGOR,KUINTAL PER HEKTAR,2015,0.00,16.29,10.77,249.14,216.00
1,2,32,JAWA BARAT,3202,KABUPATEN SUKABUMI,KUINTAL PER HEKTAR,2015,14.16,24.94,10.83,311.83,164.22
2,3,32,JAWA BARAT,3203,KABUPATEN CIANJUR,KUINTAL PER HEKTAR,2015,12.30,17.14,10.73,239.30,140.26
3,4,32,JAWA BARAT,3204,KABUPATEN BANDUNG,KUINTAL PER HEKTAR,2015,17.47,10.15,0.00,163.03,146.00
4,5,32,JAWA BARAT,3205,KABUPATEN GARUT,KUINTAL PER HEKTAR,2015,16.43,19.38,12.30,245.53,154.64
...,...,...,...,...,...,...,...,...,...,...,...,...
184,185,32,JAWA BARAT,3275,KOTA BEKASI,KUINTAL PER HEKTAR,2021,0.00,0.00,0.00,319.89,0.00
185,186,32,JAWA BARAT,3276,KOTA DEPOK,KUINTAL PER HEKTAR,2021,0.00,15.32,0.00,256.52,257.00
186,187,32,JAWA BARAT,3277,KOTA CIMAHI,KUINTAL PER HEKTAR,2021,0.00,13.22,0.00,290.77,135.08
187,188,32,JAWA BARAT,3278,KOTA TASIKMALAYA,KUINTAL PER HEKTAR,2021,11.65,15.20,0.00,309.03,231.16


In [115]:
dataset_produksi = wrangling_data('produksi')
dataset_produksi

,id,kode_provinsi,nama_provinsi,kode_kabupaten_kota,nama_kabupaten_kota,satuan,tahun,produksi_ubi_kayu,produksi_ubi_jalar,produksi_kedelai,produksi_kacang_tanah,produksi_kacang_hijau
0,53,32,JAWA BARAT,3201,KABUPATEN BOGOR,TON,2015,132519,72144,0,1445,14
1,54,32,JAWA BARAT,3202,KABUPATEN SUKABUMI,TON,2015,232715,14665,9365,12223,1132
2,55,32,JAWA BARAT,3203,KABUPATEN CIANJUR,TON,2015,147842,12708,9905,17239,339
3,56,32,JAWA BARAT,3204,KABUPATEN BANDUNG,TON,2015,89553,21330,760,1068,0
4,57,32,JAWA BARAT,3205,KABUPATEN GARUT,TON,2015,629404,98444,21433,32551,1326
...,...,...,...,...,...,...,...,...,...,...,...,...
184,238,32,JAWA BARAT,3275,KOTA BEKASI,TON,2021,896,0,0,0,0
185,239,32,JAWA BARAT,3276,KOTA DEPOK,TON,2021,513,103,0,11,0
186,240,32,JAWA BARAT,3277,KOTA CIMAHI,TON,2021,1076,149,0,19,0
187,241,32,JAWA BARAT,3278,KOTA TASIKMALAYA,TON,2021,2843,393,21,146,0


In [116]:
dataset_luas_areal_tanam = wrangling_data('luas_areal_tanam')
dataset_luas_areal_tanam

,id,kode_provinsi,nama_provinsi,kode_kabupaten_kota,nama_kabupaten_kota,satuan,tahun,luas_areal_tanam_kacang_hijau,luas_areal_tanam_ubi_kayu,luas_areal_tanam_ubi_jalar,luas_areal_tanam_kedelai,luas_areal_tanam_kacang_tanah
0,1,32,JAWA BARAT,3201,KABUPATEN BOGOR,HEKTAR,2015,13,4494,3306,89,844
1,2,32,JAWA BARAT,3202,KABUPATEN SUKABUMI,HEKTAR,2015,1045,7850,865,8075,4926
2,3,32,JAWA BARAT,3203,KABUPATEN CIANJUR,HEKTAR,2015,316,6503,848,8588,10048
3,4,32,JAWA BARAT,3204,KABUPATEN BANDUNG,HEKTAR,2015,0,5168,1481,422,1055
4,5,32,JAWA BARAT,3205,KABUPATEN GARUT,HEKTAR,2015,1128,24691,6274,13401,16805
...,...,...,...,...,...,...,...,...,...,...,...,...
184,185,32,JAWA BARAT,3275,KOTA BEKASI,HEKTAR,2021,0,32,0,0,0
185,186,32,JAWA BARAT,3276,KOTA DEPOK,HEKTAR,2021,0,16,6,0,9
186,187,32,JAWA BARAT,3277,KOTA CIMAHI,HEKTAR,2021,0,60,12,0,13
187,188,32,JAWA BARAT,3278,KOTA TASIKMALAYA,HEKTAR,2021,0,63,16,101,96


In [117]:
dataset_luas_panen = wrangling_data('luas_panen')
dataset_luas_panen

,id,kode_provinsi,nama_provinsi,kode_kabupaten_kota,nama_kabupaten_kota,satuan,tahun,luas_panen_kedelai,luas_panen_kacang_tanah,luas_panen_kacang_hijau,luas_panen_ubi_kayu,luas_panen_ubi_jalar
0,1,32,JAWA BARAT,3201,KABUPATEN BOGOR,HEKTAR,2015,0,887,13,5319,3340
1,2,32,JAWA BARAT,3202,KABUPATEN SUKABUMI,HEKTAR,2015,6614,4901,1045,7463,893
2,3,32,JAWA BARAT,3203,KABUPATEN CIANJUR,HEKTAR,2015,8052,10056,316,6178,906
3,4,32,JAWA BARAT,3204,KABUPATEN BANDUNG,HEKTAR,2015,435,1052,0,5493,1461
4,5,32,JAWA BARAT,3205,KABUPATEN GARUT,HEKTAR,2015,13045,16798,1078,25635,6366
...,...,...,...,...,...,...,...,...,...,...,...,...
184,185,32,JAWA BARAT,3275,KOTA BEKASI,HEKTAR,2021,0,0,0,28,0
185,186,32,JAWA BARAT,3276,KOTA DEPOK,HEKTAR,2021,0,7,0,20,4
186,187,32,JAWA BARAT,3277,KOTA CIMAHI,HEKTAR,2021,0,14,0,37,11
187,188,32,JAWA BARAT,3278,KOTA TASIKMALAYA,HEKTAR,2021,18,96,0,92,17


In [118]:
# eksport all dataset to csv
dataset_produktivitas.to_csv('dataset_produktivitas.csv', index=False)
dataset_produksi.to_csv('dataset_produksi.csv', index=False)
dataset_luas_areal_tanam.to_csv('dataset_luas_areal_tanam.csv', index=False)
dataset_luas_panen.to_csv('dataset_luas_panen.csv', index=False)

In [119]:
# concat all dataset and remove satuan column
cwd = os.getcwd()
all_dataset = pd.DataFrame()

for file in os.listdir(cwd):
  if file.endswith('.csv') and file.startswith('dataset'):
    data = pd.read_csv(file)
    if all_dataset.empty:
      all_dataset = data
    else:
      all_dataset = pd.concat([all_dataset, data.iloc[:,7:]], axis=1)

all_dataset.to_csv('all_dataset.csv', index=False)